In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import ffmpeg
import os
import json


from MatplotVids.data.input import ReadData
from MatplotVids.data.processing import FigureCreator
from MatplotVids.plots.circle import PlaceCircles, PlotCircles


from matplotlib.patches import Rectangle
import matplotlib.image as image
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox

from collections import Counter
import gc

In [2]:
df = pd.read_csv('../ClinVarCharts/ClinVar_2023-03.tsv',sep='\t')
df['submissiondate'] = pd.to_datetime(df['submit date'])    
df.set_index('submissiondate',inplace=True)
df = df.dropna(subset=['evaluation'])
ev = [x for x in set(df['evaluation']) if 'patho' in x.lower()]
df = df[df['evaluation'].isin(ev)]

In [5]:
xx= Counter(df['gene'])
xx = dict(sorted(xx.items(), key=lambda x:x[1],reverse=True))
genes = list(xx.keys())[:400]

In [6]:
df = df[df['gene'].isin(genes)]
df = df[['gene']]

In [7]:
x = FigureCreator.FigureScheduler(df,CatColumn='gene',ResolutionFactor=.01,MinAppearance=200,GroupingFreq='1M')


In [8]:
im = image.imread('image10.png')

In [9]:
def CreateBarPlot(d,p,ax,ColorDict,lim=30,limdown = -1.5,fontsize=8,heigth=.8):
    for n,(key,values) in enumerate(d.items()):
        if n>lim:
            continue
        y = p[key]
        if n<lim:
            ax.add_patch(Rectangle((0, y), d[key], heigth,color = ColorDict[key]))
        nText = math.ceil(d[key])
        if y>=limdown:
            ax.text(d[key],y+(heigth/2),f'{key}',ha='right',va='center',fontsize=fontsize,fontweight='bold')
        
        
        

In [ ]:
ColorDict = {}
for _ in genes:
    ColorDict[_]= PlotCircles.randomColor()


with open('colors.json','w') as j:
    json.dump(ColorDict,j)

In [10]:
with open('colors.json') as json_file:
    ColorDict = json.load(json_file)

In [4]:
CurrentPositions = {}
NumberOfBars = 25
StepSize = .2
figsize = (10,8)
stillMoving = 0
outf = 'movingbars_allClin'
outn = 'mvBars'
tit = 'pathogenic variant submissions ClinVar'
facecolor = 'lavenderblush'

In [11]:


for (frameN,frame),Ti in tqdm(zip(x.FramePlotDict.items(),x.TimeArray),total=len(x.FramePlotDict)):
    frame = {k:v for k,v in frame.items() if v>0}
    frame = dict(sorted(frame.items(), key=lambda x:x[1],reverse=True))
    maxV = max(frame.values())+5
    
    stillMoving = 0
    
    for n,entry in enumerate(frame.keys()):
        if n < NumberOfBars+5:
            wantedPos = float(NumberOfBars - n)
        else:
            wantedPos = -2
        if entry not in ColorDict:
            ColorDict[entry]= PlotCircles.randomColor()
        if entry not in CurrentPositions:
            CurrentPositions[entry] = round(float(-2),2)
        
        CurrentPos = CurrentPositions[entry]
        if CurrentPos > wantedPos:
            CurrentPositions[entry] = round(CurrentPos-StepSize,2)
            stillMoving = 1
        elif CurrentPos < wantedPos:
            CurrentPositions[entry] = round(CurrentPos+StepSize,2)
            stillMoving = 1
        
    cc=len(str(frameN))
    cc=(5-cc)*'0'+str(frameN)
    if frameN<0:
        continue
    if os.path.exists(f'{outf}//{outn}_{cc}.jpg'):
        continue
    
    
    fig,ax = plt.subplots(figsize=figsize)

    CreateBarPlot(frame,CurrentPositions,ax,ColorDict)
    ax.set(ylim=(-1,NumberOfBars+1),xlim=(0,maxV),title=f'{tit}-{Ti.year}-{Ti.month}',xlabel='submissions per gene')
    ax.set_yticks([],[])

    imagebox = OffsetImage(im, zoom=.4, alpha=0.2)
    imagebox.image.axes = ax
    ao = AnchoredOffsetbox(4, pad=0.01, borderpad=0, child=imagebox)
    ao.patch.set_alpha(0)
    ax.add_artist(ao)
    plt.subplots_adjust(left=0.05, right=0.99, top=0.95, bottom=0.05)
    plt.savefig(f'{outf}//{outn}_{cc}.jpg',facecolor=facecolor,dpi=150)
    plt.clf()
    plt.close("all")
    gc.collect()

    
    
while stillMoving == 1:
    frameN = frameN + 1
    frame = {k:v for k,v in frame.items() if v>0}
    frame = dict(sorted(frame.items(), key=lambda x:x[1],reverse=True))
    maxV = max(frame.values())+5
    
    stillMoving = 0
    
    for n,entry in enumerate(frame.keys()):
        if n < NumberOfBars+5:
            wantedPos = float(NumberOfBars - n)
        else:
            wantedPos = -2
        if entry not in ColorDict:
            ColorDict[entry]= PlotCircles.randomColor()
        if entry not in CurrentPositions:
            CurrentPositions[entry] = round(float(-2),2)
        
        CurrentPos = CurrentPositions[entry]
        if CurrentPos > wantedPos:
            CurrentPositions[entry] = round(CurrentPos-StepSize,2)
            stillMoving = 1
        elif CurrentPos < wantedPos:
            CurrentPositions[entry] = round(CurrentPos+StepSize,2)
            stillMoving = 1
        
    
    fig,ax = plt.subplots(figsize=figsize)

    CreateBarPlot(frame,CurrentPositions,ax,ColorDict)
    ax.set(ylim=(-1,NumberOfBars+1),xlim=(0,maxV),title=f'{tit}-{Ti.year}-{Ti.month}',xlabel='submissions per gene')
    ax.set_yticks([],[])
    
    cc=len(str(frameN))
    cc=(5-cc)*'0'+str(frameN)
    
    imagebox = OffsetImage(im, zoom=.4, alpha=0.2)
    imagebox.image.axes = ax
    ao = AnchoredOffsetbox(4, pad=0.01, borderpad=0, child=imagebox)
    ao.patch.set_alpha(0)
    ax.add_artist(ao)
    plt.subplots_adjust(left=0.05, right=0.99, top=0.95, bottom=0.05)

    plt.savefig(f'{outf}//{outn}_{cc}.jpg',facecolor=facecolor,dpi=150)
    plt.close()
    plt.close()

    
for _ in range(96):
    frameN = frameN + 1
    
    fig,ax = plt.subplots(figsize=figsize)

    CreateBarPlot(frame,CurrentPositions,ax,ColorDict)

    ax.set(ylim=(-1,NumberOfBars+1),xlim=(0,maxV),title=f'{tit}-{Ti.year}-{Ti.month}',xlabel='submissions per gene')
    ax.set_yticks([],[])
    cc=len(str(frameN))
    cc=(5-cc)*'0'+str(frameN)
    imagebox = OffsetImage(im, zoom=.4, alpha=0.2)
    imagebox.image.axes = ax
    ao = AnchoredOffsetbox(4, pad=0.01, borderpad=0, child=imagebox)
    ao.patch.set_alpha(0)
    ax.add_artist(ao)
    plt.subplots_adjust(left=0.05, right=0.99, top=0.95, bottom=0.05)

    plt.savefig(f'{outf}//{outn}_{cc}.jpg',facecolor=facecolor,dpi=150)
    plt.close()
    plt.close()
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14653/14653 [36:34<00:00,  6.68it/s]


In [6]:
ls '../02_vid/MatplotVids/movingbars_allClin/'

mvBars_00000.jpg  mvBars_03688.jpg  mvBars_07376.jpg  mvBars_11064.jpg
mvBars_00001.jpg  mvBars_03689.jpg  mvBars_07377.jpg  mvBars_11065.jpg
mvBars_00002.jpg  mvBars_03690.jpg  mvBars_07378.jpg  mvBars_11066.jpg
mvBars_00003.jpg  mvBars_03691.jpg  mvBars_07379.jpg  mvBars_11067.jpg
mvBars_00004.jpg  mvBars_03692.jpg  mvBars_07380.jpg  mvBars_11068.jpg
mvBars_00005.jpg  mvBars_03693.jpg  mvBars_07381.jpg  mvBars_11069.jpg
mvBars_00006.jpg  mvBars_03694.jpg  mvBars_07382.jpg  mvBars_11070.jpg
mvBars_00007.jpg  mvBars_03695.jpg  mvBars_07383.jpg  mvBars_11071.jpg
mvBars_00008.jpg  mvBars_03696.jpg  mvBars_07384.jpg  mvBars_11072.jpg
mvBars_00009.jpg  mvBars_03697.jpg  mvBars_07385.jpg  mvBars_11073.jpg
mvBars_00010.jpg  mvBars_03698.jpg  mvBars_07386.jpg  mvBars_11074.jpg
mvBars_00011.jpg  mvBars_03699.jpg  mvBars_07387.jpg  mvBars_11075.jpg
mvBars_00012.jpg  mvBars_03700.jpg  mvBars_07388.jpg  mvBars_11076.jpg
mvBars_00013.jpg  mvBars_03701.jpg  mvBars_07389.jpg  mvBars_11077.jpg
mvBars

mvBars_02872.jpg  mvBars_06560.jpg  mvBars_10248.jpg  mvBars_13936.jpg
mvBars_02873.jpg  mvBars_06561.jpg  mvBars_10249.jpg  mvBars_13937.jpg
mvBars_02874.jpg  mvBars_06562.jpg  mvBars_10250.jpg  mvBars_13938.jpg
mvBars_02875.jpg  mvBars_06563.jpg  mvBars_10251.jpg  mvBars_13939.jpg
mvBars_02876.jpg  mvBars_06564.jpg  mvBars_10252.jpg  mvBars_13940.jpg
mvBars_02877.jpg  mvBars_06565.jpg  mvBars_10253.jpg  mvBars_13941.jpg
mvBars_02878.jpg  mvBars_06566.jpg  mvBars_10254.jpg  mvBars_13942.jpg
mvBars_02879.jpg  mvBars_06567.jpg  mvBars_10255.jpg  mvBars_13943.jpg
mvBars_02880.jpg  mvBars_06568.jpg  mvBars_10256.jpg  mvBars_13944.jpg
mvBars_02881.jpg  mvBars_06569.jpg  mvBars_10257.jpg  mvBars_13945.jpg
mvBars_02882.jpg  mvBars_06570.jpg  mvBars_10258.jpg  mvBars_13946.jpg
mvBars_02883.jpg  mvBars_06571.jpg  mvBars_10259.jpg  mvBars_13947.jpg
mvBars_02884.jpg  mvBars_06572.jpg  mvBars_10260.jpg  mvBars_13948.jpg
mvBars_02885.jpg  mvBars_06573.jpg  mvBars_10261.jpg  mvBars_13949.jpg
mvBars

In [8]:
from glob import glob
import shutil

In [16]:
for _ in glob('../02_vid/MatplotVids/movingbars_allClin/*'):
    shutil.copyfile(_,f'_temp/{_.split("/")[-1]}')

In [ ]:
shutil.copyfile

In [20]:
(
    ffmpeg
    .input(f'_temp/*.jpg', pattern_type='glob', framerate=150,)
    .output('movingbarsLow.gif')
    .run()
)

ffmpeg version 5.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-16)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutl

[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 8, current: 6; changing to 9. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 9, current: 7; changing to 10. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 10, current: 7; changing to 11. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 11, current: 8; changing to 12. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 12, current: 9; changing to 13. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 13, current: 9; changing to 14. This may result in incorrect timestamps in the output file.
[gif @ 

[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 61, current: 41; changing to 62. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 62, current: 42; changing to 63. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 63, current: 43; changing to 64. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 64, current: 43; changing to 65. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 65, current: 44; changing to 66. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 66, current: 45; changing to 67. This may result in incorrect timestamps in the output file

[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 116, current: 78; changing to 117. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 117, current: 79; changing to 118. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 118, current: 79; changing to 119. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 119, current: 80; changing to 120. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 120, current: 81; changing to 121. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 121, current: 81; changing to 122. This may result in incorrect timestamps in the

[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 166, current: 111; changing to 167. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 167, current: 112; changing to 168. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 168, current: 113; changing to 169. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 169, current: 113; changing to 170. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 170, current: 114; changing to 171. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 171, current: 115; changing to 172. This may result in incorrect timestamps 

[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 219, current: 147; changing to 220. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 220, current: 147; changing to 221. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 221, current: 148; changing to 222. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 222, current: 149; changing to 223. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 223, current: 149; changing to 224. This may result in incorrect timestamps in the output file.
[gif @ 0x55a8dae9b580] Non-monotonous DTS in output stream 0:0; previous: 224, current: 150; changing to 225. This may result in incorrect timestamps 

KeyboardInterrupt: 

In [15]:
(
    ffmpeg
    .input(f'{outf}/*.jpg', pattern_type='glob', framerate=112)
    .output('movingbars.mp4')
    .run()
)

ffmpeg version 5.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-16)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1660333770726/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutl

(None, None)